In [1]:
from hough_functions import LineDetector
from ultralytics import YOLO
import csv
import cv2
import math

In [2]:
def crop_image(image):
    '''
    :param image: An image imported through cv2.imread()
    :return: Cropped image according to rules specified within function.
    '''
    # Get the dimensions of the image
    height, width = image.shape[:2]

    # Define the region of interest (ROI)
    roi_x = 2 * width // 5  # Starting x-coordinate of the ROI (left boundary of the middle fifth)
    roi_width = width // 5  # Width of the ROI (1/5 of the width of the image)

    # Define the bottom 2/3 of the ROI
    bottom = height # The height of the image = the bottom coordinate
    bottom_roi_height = height//2  # Height of the bottom half

    # Crop the region of interest from the image
    roi = image[bottom_roi_height:bottom, roi_x:roi_x+roi_width]

    return roi, roi_x, bottom_roi_height
    
def detect_lines_image(image_path, crop=True):
    image = cv2.imread(image_path)
    x_adj = 0
    y_adj = 0
    if image is None:
        print("Error opening image")
        return
    elif crop:
        image, x_tmp, y_tmp = crop_image(image)
        x_adj += x_tmp
        y_adj += y_tmp
    processed_image, lines = process_frame(image)
    height, width, _ = image.shape
    for line in lines:
        line[0][0] += x_adj
        line[0][1] += y_adj
        line[0][2] += x_adj
        line[0][3] += y_adj
        x1, y1, x2, y2 = line[0]
        if x2 - x1 != 0:  # Avoid division by zero
            slope = (y2 - y1) / (x2 - x1)
            intercept = y1 - slope * x1
            print(f"Line Detected with equation x = (y - {y1}) / {slope} + {x1}")
        else:
            print(f"Vertical Line Detected at x = {x1}")
    cv2.imwrite('hough_output_image.jpg', processed_image)
    return lines

def detect_lines_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video stream or file")
        return
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    out = cv2.VideoWriter('hough_output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 10, (frame_width, frame_height))
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            processed_frame = process_frame(frame)
            out.write(processed_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()

def process_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    lines = cv2.HoughLinesP(edges, 1, math.pi / 180, 100, minLineLength=200, maxLineGap=10)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return frame, lines

In [69]:
lines = detect_lines_image('skyline_icecream_truck.png', crop=True)

Line Detected with equation x = (y - 992) / -3.769230769230769 + 826
Line Detected with equation x = (y - 746) / 3.4871794871794872 + 1016


In [70]:
lines

array([[[ 826,  992,  904,  698]],

       [[1016,  746, 1094, 1018]]], dtype=int32)

In [71]:
x1 = lines[1][0][0]
y1 = lines[1][0][1]
x2 = lines[1][0][2]
y2 = lines[1][0][3]

In [72]:
y = 620

In [73]:
slope = (y2 - y1) / (x2 - x1)
x = (y - y1)/slope + x1
x

979.8676470588235

In [74]:
def calculate_x_val(line, y):
    '''
    :param line: Coordinates [[x1, y1, x2, y2]] that define a line.
    :param y: Y value of the bottom of a detected object.
    :return: X value where object intercepts the line.
    '''
    x1 = line[0][0]
    y1 = line[0][1]
    x2 = line[0][2]
    y2 = line[0][3]
    slope = (y2 - y1) / (x2 - x1)
    x = (y - y1)/slope + x1
    return x

In [77]:
calculate_x_val(lines[0], y)

924.6938775510204